Aqui temos a instalação das bibliotecas necessárias

In [1]:
!pip install crewai crewai_tools langchain_community
import warnings
warnings.filterwarnings('ignore')

Importando as classes Agent, Task e Crew, as quais são respectivamente responsáveis por nos permitir criar nossos agentes, criar as tarefas que nossos agentes vão desempenhar e por fim realizar a orquestração. LLM foi inserido pois não estou conseguindo usar meus créditos da OpenAI.

In [2]:
from crewai import Agent, Task, Crew, LLM

LLM que vai se conectar aos nosso agente e credenciais, pelo fato de não conseguir usar a da OpenAI vou usar minhas credenciais do gemini da google e devido a isso vou inserir, o parâmetro llm em todos os agentes para que eles usem o modelo gemini.

In [3]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.environ["GEMINI_API_KEY"]
            )

Abaixo temos nosso primeiro agente o pesquisador_mercado_financeiro, o qual conta com a classe Agent definida anteriormente, a qual conta com os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é pesquisar sobre 3 tópicos que podem ser manipulados pelo usuário relacionados a economia e uma história de fundo que gera contexto para o agente. Além disso temos os parâmetros allow_delegation e verbose, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [4]:
pesquisador_mercado_financeiro = Agent(
    role="Pesquisador de Notícias do Mercado Financeiro",
    goal="Procurar notícias associadas a {palavra_chave_1}, {palavra_chave_2} e {palavra_chave_3}",
    backstory="Você é um economista-chefe da B3,"
              "que busca notícias relacionadas aos seguintes tópicos: {palavra_chave_1}, {palavra_chave_2} e {palavra_chave_3}."
              "Essas notíciias são pesquisadas dos principais portais financeiros."
              "Você coleta para cada notícia pesquisada {coluna_1}, {coluna_2}, {coluna_3} e {coluna_4}"
              "para que seja produzido uma tabela contendo as principais informações de cada notícia,"
              "a qual vai ser usado para alimentar um banco de dados."
              "Seu trabalho é procurar essas notícias.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

Abaixo temos nosso segundo agente o analista_derivativos, o qual é bastante similar ao primeiro pois conta com a classe Agent definida, a qual possui os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é pegar as notícias pesquisadas pelo primeiro agente e com base nisso procura ativos do mercado de derivativos já o backstory se trata de uma história de fundo que gera contexto para o agente detalhando mais como ele deve agir e se portar. Além disso temos os parâmetros allow_delegation e verbose assim como no agente anterior, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [5]:
analista_derivativos = Agent(
    role="Analista de Derivativos",
    goal="Identificar oportunidades no mercado de derivativos com base em notícias financeiras.",
    backstory=(
        "Um especialista em mercados financeiros com vasta experiência em análise "
        "de derivativos como futuros, opções e swaps. Seu trabalho é identificar padrões, "
        "avaliar riscos e recomendar os melhores ativos baseados em eventos de mercado."
    ),
    verbose=True,
    llm=llm
)

Abaixo temos nosso terceiro agente o economista_brasileiro, o qual é bastante similar aos demais agentes, pois conta com a classe Agent definida, a qual possui os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é pegar os ativos que o segundo agente selecionou com base nas notícias que recebeu pelo primeiro agente e com base produz um relatório relacionando isso com a economia brasileira já o backstory se trata de uma história de fundo que gera contexto para o agente detalhando mais como ele deve agir e se portar. Além disso temos os parâmetros allow_delegation e verbose assim como no agente anterior, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [6]:
economista_brasileiro = Agent(
    role="Especialista em Economia Brasileira",
    goal="Analisar os principais eventos econômicos brasileiros e seu impacto nos ativos financeiros, fornecendo insights detalhados para investidores.",
    backstory=(
        "Sou um economista experiente, especializado no mercado financeiro brasileiro. "
        "Com anos de análise macroeconômica e conhecimento profundo sobre a política monetária, fiscal e cambial do Brasil, "
        "eu traduzo dados e eventos econômicos em insights acionáveis para investidores e empresas."
    ),
    verbose=True,
    memory=True,
    llm=llm
)

Abaixo temos a a variável coletar_noticias_task, a qual é uma das tasks que nossos agentes podem realizar, dessa forma temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, sendo eles: buscar notícias relacionadas a 3 tópicos e usar eles como  palvras chaves na pesquisa onde produzimos uma tabela com 4 variáveis de escolha, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual deve ser essa tabela onde há cada notícia com representando uma linha e cada variável uma coluna, uma tabela, e o parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o planner criadfo anteriormente.

In [7]:
coletar_noticias_task = Task(
    description=(
        "Você deve pesquisar notícias financeiras nos principais portais especializados, "
        "buscando informações relacionadas aos seguintes tópicos: "
        "**{palavra_chave_1}, {palavra_chave_2} e {palavra_chave_3}**.\n\n"
        "Para cada notícia encontrada, extraia e registre as seguintes informações:\n"
        " **{coluna_1}**\n"
        " **{coluna_2}**\n"
        " **{coluna_3}**\n"
        " **{coluna_4}**\n\n"
        "**Critérios importantes:**\n"
        "- Priorize fontes **confiáveis** (ex: Valor Econômico, Bloomberg, Reuters, etc.).\n"
        "- Certifique-se de que as notícias sejam **recentes** (últimos 7 dias, se possível).\n"
        "- Se houver notícias duplicadas, mantenha apenas a mais completa e relevante.\n\n"
        "**Objetivo:**\n"
        "Gerar uma tabela estruturada contendo todas as notícias coletadas, garantindo que os dados "
        "possam ser facilmente utilizados para alimentar um banco de dados ou relatório financeiro."
    ),
    expected_output=(
        "Uma tabela bem estruturada com informações organizadas, incluindo colunas "
        "**{coluna_1}, {coluna_2}, {coluna_3}, {coluna_4}**, contendo apenas notícias "
        "relevantes e confiáveis."
    ),
    agent=pesquisador_mercado_financeiro
)

Abaixo temos a a variável tarefa_analise_derivativos, a qual é uma das tasks que nossos agentes podem realizar e assim como a anterior temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, e output esperado é uma lista com 3 ativos relacionados as notícias que ele consumiu conforme coloca a task, já parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o analista_derivativos criado anteriormente.

In [8]:
tarefa_analise_derivativos = Task(
    description=(
        "Analise as notícias financeiras mais recentes para identificar tendências e oportunidades "
        "no mercado de derivativos. Baseado nas informações, selecione pelo menos três ativos derivativos "
        "(como contratos futuros, opções ou swaps) e justifique sua escolha considerando fatores como volatilidade, "
        "indicadores econômicos e notícias relevantes."
    ),
    expected_output="Uma lista de 3 ativos derivativos recomendados, com uma breve análise justificando cada escolha.",
    agent=analista_derivativos
)

E por fim abaixo temos a variável analise_economica_task, a qual é uma das tasks que nossos agentes podem realizar e assim como a anterior temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, onde a idéia central é realizar uma análise econômica detalhada relacionando o contexto econômico brasileiro, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual pode ser um relatório, já parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o economista_brasileiro criado anteriormente.

In [9]:
analise_economica_task = Task(
    description=(
        "Realize uma análise detalhada dos principais eventos econômicos brasileiros recentes, "
        "incluindo decisões do Banco Central, políticas fiscais, variação do PIB e mudanças regulatórias. "
        "Explique como esses fatores influenciaram ativos financeiros como ações, títulos públicos e câmbio. "
        "Forneça uma conclusão clara e recomendações estratégicas."
    ),
    expected_output=(
        "Um relatório em formato de artigo com pelo menos 500 palavras, abordando: "
        "- Os principais eventos econômicos recentes no Brasil. "
        "- O impacto desses eventos sobre ações, renda fixa, câmbio e outros ativos financeiros. "
        "- Uma conclusão com insights para investidores."
    ),
    agent=economista_brasileiro
)

Abaixo temos a variável crew, a qual recebe a classe Crew, onde temos os seguintes parâmetros agents, tasks e verbose, os quais respectivamente se tratam de uma lista de agentes os quais vão executar as tarefas, a lista de tarefas a serem executadas por nossos agentes e os pensamentos de nossos agentes ao executar as tarefas. Além vale destacar que nesse caso as tarefas são executadas sequencialmente, isto é plan -> write -> edit. Aqui diferente do tutorial a verbose teve que ser ajustado para true, devido as versões mais recentes da biblioteca.

In [10]:
crew = Crew(
    agents=[pesquisador_mercado_financeiro, analista_derivativos, economista_brasileiro],
    tasks=[coletar_noticias_task, tarefa_analise_derivativos, analise_economica_task],
    verbose=True
)

Abaixo temos a variável results a qual recebe o metódo kickoff da variável crew, onde temos os inputs que se tratam de valores interpolados que podemos substituir nos agentes, ou seja, para todas as vezes que apraceu a palavra topic tanto na definição dos agentes ou das tasks podemos substituir por Artificial Intelligence e ao executar podemos observar a execução de cada tarefa uma por uma devido ao fato de verbose estar ativa, como declarado anteriormente.

In [11]:
result = crew.kickoff(inputs={
    "palavra_chave_1": "macroeconomia brasileira",
    "palavra_chave_2": "economia brasileira",
    "palavra_chave_3": "mercado brasileiro",
    "coluna_1": "dados",
    "coluna_2": "autor",
    "coluna_3": "resumo",
    "coluna_4": "fontes"
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6909ce7d-2cfc-4c34-a015-18a9e8690963                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6f7312ab-4337-4c23-89a4-aa3766e233c9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6f7312ab-4337-4c23-89a4-aa3766e233c9
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
            Status: In Progress

# Agent: Pesquisador de Notícias do Mercado Financeiro
## Task: Você deve pesquisar notícias financeiras nos principais portais especializados, buscando informações relacionadas aos seguintes tópicos: **macroeconomia brasileira, economia brasileira e mercado brasileiro**.

Para cada notícia encontrada, extraia e registre as seguintes informações:
 **dados**
 **autor**
 **resumo**
 **fontes**

**Critérios importantes:**
- Priorize fontes **confiáveis** (ex: Valor Econômico, Bloomberg, Reuters, etc.).
- Certifique-se de que as notícias sejam **recentes** (últimos 7 dias, se possível).
- Se houver notícias duplicadas, mantenha apenas a mais completa e relevante.

**Objetivo:**
Gerar uma tabela estruturada contendo todas as notícias coletadas, garantindo que os dados possam ser facilmente utilizados para alimentar um banco de dados ou relatório financeiro.


🚀 Crew: crew
└── 📋 Task: 6f7312ab-4337-4c23-89a4-aa3766e233c9
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Especialista em Economia Brasileira
## Final Answer: 
**Análise dos Principais Eventos Econômicos Recentes no Brasil e seu Impacto nos Ativos Financeiros**


**Introdução:**

O cenário econômico brasileiro recente tem sido marcado por uma série de eventos complexos e interligados, influenciando significativamente o desempenho de ativos financeiros.  Esta análise se baseia em um cenário hipotético, mas espelha tendências e eventos plausíveis, explorando o impacto sobre ações, títulos públicos, câmbio e outros ativos.


**Cenário Hipotético:**

Imagine que nos últimos meses, o Brasil vivenciou os seguintes eventos:


* **1.  Aumento da Taxa Selic (Hipotético):**  O Banco Central, preocupado com a persistente inflação acima da meta, elevou a taxa Selic em 0.5% (de 13,25% para 13,75%), apesar de sinais de desaceleração da economia.  (Fonte hipotética: Valor Econômico - 26/10/2024).
* **2.  Revisão para Baixo do Crescimento do PIB (Hipotético):**  O IBGE revisou suas projeções de

🚀 Crew: crew
├── 📋 Task: 6f7312ab-4337-4c23-89a4-aa3766e233c9
│      Assigned to: Pesquisador de Notícias do Mercado Financeiro
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
│           Status: ✅ Completed
├── 📋 Task: 0357638a-3923-4888-ae1f-45c6043ac97a
│      Assigned to: Analista de Derivativos
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista de Derivativos
│           Status: ✅ Completed
└── 📋 Task: 7f333434-3dd7-4420-9139-4b34bd1de02e
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Economia Brasileira
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 6f7312ab-4337-4c23-89a4-aa3766e233c9
│      Assigned to: Pesquisador de Notícias do Mercado Financeiro
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
│           Status: ✅ Completed
├── 📋 Task: 0357638a-3923-4888-ae1f-45c6043ac97a
│      Assigned to: Analista de Derivativos
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista de Derivativos
│           Status: ✅ Completed
└── 📋 Task: 7f333434-3dd7-4420-9139-4b34bd1de02e
       Assigned to: Especialista em Economia Brasileira
       Status: ✅ Completed
    └── 🤖 Agent: Especialista em Economia Brasileira
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7f333434-3dd7-4420-9139-4b34bd1de02e                                                                     │
│  Agent: Especialista em Economia Brasileira                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6909ce7d-2cfc-4c34-a015-18a9e8690963                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

E por fim temos a exibição de tudo organizado com nosso relatório relacionando as notícias com a economia brasileira e com os ativos.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

**Análise dos Principais Eventos Econômicos Recentes no Brasil e seu Impacto nos Ativos Financeiros**


**Introdução:**

O cenário econômico brasileiro recente tem sido marcado por uma série de eventos complexos e interligados, influenciando significativamente o desempenho de ativos financeiros.  Esta análise se baseia em um cenário hipotético, mas espelha tendências e eventos plausíveis, explorando o impacto sobre ações, títulos públicos, câmbio e outros ativos.


**Cenário Hipotético:**

Imagine que nos últimos meses, o Brasil vivenciou os seguintes eventos:


* **1.  Aumento da Taxa Selic (Hipotético):**  O Banco Central, preocupado com a persistente inflação acima da meta, elevou a taxa Selic em 0.5% (de 13,25% para 13,75%), apesar de sinais de desaceleração da economia.  (Fonte hipotética: Valor Econômico - 26/10/2024).
* **2.  Revisão para Baixo do Crescimento do PIB (Hipotético):**  O IBGE revisou suas projeções de crescimento do PIB para 2024 para baixo, de 2% para 1.5%, citando incertezas políticas e um consumo privado mais fraco do que o esperado.  (Fonte hipotética: Agência Brasil - 25/10/2024).
* **3.  Aceleração da Inflação (Hipotético):** Dados de inflação referentes ao mês de outubro surpreenderam negativamente com um índice maior do que o esperado. (Fonte hipotética: Instituto Brasileiro de Geografia e Estatística - IBGE - 24/10/2024).
* **4.  Aprovação de Reforma Tributária (Hipotético):** O Congresso Nacional aprovou uma ampla reforma tributária, visando simplificar o sistema e melhorar a arrecadação.  Embora positiva a longo prazo, a reforma gera incertezas no curto prazo devido a potenciais impactos sobre os setores específicos.  (Fonte hipotética:  Folha de São Paulo - 23/10/2024).


**Impacto nos Ativos Financeiros:**

* **Ações:** O aumento da Selic e a revisão para baixo do PIB impactaram negativamente o mercado acionário.  O aumento dos juros afeta a atratividade das ações, enquanto a perspectiva de menor crescimento reduz as expectativas de lucros corporativos.  A reforma tributária, por sua vez, introduziu volatilidade, com setores específicos reagindo de forma distinta.
* **Renda Fixa:** O aumento da Selic levou a uma alta nos rendimentos dos títulos públicos, tornando-os mais atrativos para investidores.  Porém, a incerteza em relação à economia e à inflação manteve a volatilidade em títulos de prazo mais longo.
* **Câmbio:** A combinação de aumento da Selic e menor perspectiva de crescimento do PIB levou a uma valorização do real frente ao dólar.  A procura por ativos de maior segurança aumentou, favorecendo a moeda brasileira.
* **Outros Ativos:**  Ativos de risco, como commodities, também sofreram impacto.  A desaceleração econômica diminuiu a demanda por matérias-primas.

**Conclusão e Recomendações Estratégicas:**

O cenário atual é complexo, requer cautela e uma estratégia de diversificação. O aumento da Selic, embora combata a inflação, também freia o crescimento econômico. A reforma tributária, apesar dos benefícios potenciais a longo prazo, introduz incerteza no curto prazo.

**Recomendações:**

* **Renda Fixa:**  Manter posição em títulos públicos indexados à inflação (IPCA) para proteger o portfólio da inflação, e diversificar em títulos prefixados com prazos adequados ao seu perfil de risco.
* **Ações:** Selecionar ações de empresas com fundamentos sólidos e com potencial de crescimento sustentável, mesmo em um cenário de crescimento econômico mais lento.  Concentrar-se em setores menos sensíveis ao ciclo econômico.
* **Câmbio:**  Manter uma posição em dólar como hedge contra a volatilidade cambial, mas considerando a valorização do Real.

**Disclaimer:** Esta análise baseia-se em um cenário hipotético e não constitui recomendação de investimento. Investidores devem consultar seus assessores financeiros antes de tomar qualquer decisão.